In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [56]:
from nltk.corpus  import stopwords


In [2]:
!pip install kagglehub

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)


In [58]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/vadik/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [63]:
from string import punctuation

In [64]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [62]:
"'" in stopwords.words("english")

False

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anandshaw2001/netflix-movies-and-tv-shows")

print("Path to dataset files:", path)

Path to dataset files: /Users/vadik/.cache/kagglehub/datasets/anandshaw2001/netflix-movies-and-tv-shows/versions/1


In [2]:
df = pd.read_csv('/Users/vadik/.cache/kagglehub/datasets/anandshaw2001/netflix-movies-and-tv-shows/versions/1/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [55]:
df.shape

(8807, 12)

In [4]:
df.type.replace("Movie", 1, inplace=True)
df.type.replace("TV Show", 0, inplace=True)


/var/folders/2c/_cffglkn0bg2hj7q75cmkg7w0000gn/T/ipykernel_7530/3318831163.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.type.replace("Movie", 1, inplace=True)
/var/folders/2c/_cffglkn0bg2hj7q75cmkg7w0000gn/T/ipykernel_7530/3318831163.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [5]:
df.type.unique()

array([1, 0])

In [8]:
description_listik = df.description.to_list()

In [11]:
df.description.isna().sum()

0

In [16]:
all([i.strip() for i in description_listik])

True

In [17]:
" adfl;hfa ".strip()

'adfl;hfa'

In [19]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(description_listik[50]))

['After', 'the', 'ancient', 'Great', 'War', ',', 'the', 'god', 'Chitragupta', 'oversees', 'a', 'trial', 'to', 'determine', 'who', 'were', 'the', 'battle', "'", 's', 'true', 'heroes', 'and', 'villains', '.']


In [21]:
description_listik_tok = [tokenizer.tokenize(i.lower()) for i in description_listik]

In [22]:
from gensim.models import Word2Vec
model = Word2Vec(description_listik_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  

In [23]:
import gensim.downloader as api
# api.load('glove-wiki-gigaword-50')

# api.load('glove-wiki-gigaword-100')
model_glove = api.load('glove-twitter-100')

In [24]:
def get_phrase_embedding(phrase, model):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase_l  = phrase.lower()
    # print(phrase_l)
    tokens = tokenizer.tokenize(phrase_l)
    # print(tokens)
    
    words_vectors = np.array([model.get_vector(word) for word in tokens if word in model.index_to_key])
    # print(word_vectors)
    
    
    vector = np.mean(words_vectors, axis=0)
    # print(vector, vector.shape )
    if not vector.all():
        vector = np.zeros([model.vector_size], dtype='float32')
    
    
    return vector

In [ ]:
emb_word2vec = [get_phrase_embedding(i, model)for i in description_listik]

In [27]:
emb_glove = [get_phrase_embedding(i, model_glove)for i in description_listik]

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [31]:
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(
    emb_word2vec, df.type.to_list(),stratify=df.type.to_list(), test_size=0.3, random_state=42)

In [38]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(
    emb_glove, df.type.to_list(),stratify=df.type.to_list(), test_size=0.3, random_state=42)

In [50]:
reg = LogisticRegression()

In [36]:
reg_glove = LogisticRegression()

In [39]:
reg_glove.fit(X_train_glove, y_train_glove)

LogisticRegression()

In [51]:
reg.fit(X_train_word2vec, y_train_word2vec)

LogisticRegression()

In [52]:
# Predict
y_pred = reg.predict(X_test_word2vec)

# Evaluation
print("Accuracy:", accuracy_score(y_test_word2vec, y_pred))
print("\nClassification Report:\n", classification_report(y_test_word2vec, y_pred))

Accuracy: 0.7393113885735906

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.30      0.41       803
           1       0.75      0.93      0.83      1840

    accuracy                           0.74      2643
   macro avg       0.70      0.62      0.62      2643
weighted avg       0.72      0.74      0.71      2643



In [53]:
y_pred.mean()

0.8584941354521377

In [41]:
# Predict
y_pred = reg_glove.predict(X_test_glove)

# Evaluation
print("Accuracy:", accuracy_score(y_test_glove, y_pred))
print("\nClassification Report:\n", classification_report(y_test_glove, y_pred))

Accuracy: 0.7393113885735906

Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.30      0.41       803
           1       0.75      0.93      0.83      1840

    accuracy                           0.74      2643
   macro avg       0.70      0.62      0.62      2643
weighted avg       0.72      0.74      0.71      2643



In [43]:
y_pred.mean()

0.8584941354521377

In [44]:
df.type.mean()

0.6961507891449983

In [46]:
np.mean(y_train_glove)

0.6961388708630759

In [47]:
X_train_glove, X_test_glove, y_train_glove_without, y_test_glove = train_test_split(
    emb_glove, df.type.to_list(), test_size=0.3, random_state=42)

In [48]:
np.mean(y_train_glove_without)

0.6967878001297858

In [56]:
df.type.unique()

array([0, 1])

In [55]:
df.type = df.type.replace({'Movie': 0, 'TV Show':1})

/var/folders/2c/_cffglkn0bg2hj7q75cmkg7w0000gn/T/ipykernel_2451/3853251246.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.type = df.type.replace({'Movie': 0, 'TV Show':1})


In [ ]:
df.type = df.type.replace('Movie', 0)

In [57]:
df_description = df.description.to_list()

data_for_word2vec = df_description

In [58]:
len(df_description)

8807

In [59]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(df_description[0]))

['As', 'her', 'father', 'nears', 'the', 'end', 'of', 'his', 'life', ',', 'filmmaker', 'Kirsten', 'Johnson', 'stages', 'his', 'death', 'in', 'inventive', 'and', 'comical', 'ways', 'to', 'help', 'them', 'both', 'face', 'the', 'inevitable', '.']


In [60]:
data_for_word2vec_tok = [tokenizer.tokenize(i) for i in data_for_word2vec]

In [61]:
df_description_tok = [tokenizer.tokenize(i) for i in df_description]

In [62]:
from gensim.models import Word2Vec
model = Word2Vec(data_for_word2vec_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv 

In [41]:
model.most_similar('bear')

[('violent', 0.9954766035079956),
 ('little', 0.9949995875358582),
 ('unexpectedly', 0.9949324131011963),
 ('tragic', 0.9948938488960266),
 ('singer', 0.9948647022247314),
 ('street', 0.9946455955505371),
 ('string', 0.9942256212234497),
 ('makes', 0.9938539862632751),
 ('journalist', 0.9938167333602905),
 ('case', 0.993737518787384)]

In [75]:
def get_phrase_embedding(phrase, model):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase_l  = phrase.lower()
    # print(phrase_l)
    tokens = tokenizer.tokenize(phrase_l)
    # print(tokens)
    
    words_vectors = np.array([model.get_vector(word) for word in tokens if word in model.index_to_key])
    # print(word_vectors)
    
    
    vector = np.mean(words_vectors, axis=0)
    # print(vector, vector.shape )
    if not vector.all():
        vector = np.zeros([model.vector_size], dtype='float32')
    
    
    return vector

In [ ]:
description_emb = [get_phrase_embedding(phrase, model) for phrase in data_for_word2vec]

In [67]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


In [68]:
X_train, X_test, y_train, y_test = train_test_split(description_emb, df.type.to_list(), test_size=0.2, random_state=42)


In [71]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [72]:
# Predict
y_pred = clf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.706583427922815

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.98      0.82      1214
           1       0.68      0.11      0.18       548

    accuracy                           0.71      1762
   macro avg       0.70      0.54      0.50      1762
weighted avg       0.70      0.71      0.62      1762



In [73]:
import gensim.downloader as api
# api.load('glove-wiki-gigaword-50')

# api.load('glove-wiki-gigaword-100')
model_glove = api.load('glove-twitter-100')

In [83]:
description_emb_glove = [get_phrase_embedding(phrase, model_glove) for phrase in data_for_word2vec]

In [84]:
X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(description_emb_glove, df.type.to_list(), test_size=0.2, random_state=42)


In [85]:
clf = LogisticRegression(random_state=0).fit(X_train_glove, y_train)

In [86]:
# Predict
y_pred = clf.predict(X_test_glove)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7440408626560726

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.93      0.83      1214
           1       0.68      0.33      0.45       548

    accuracy                           0.74      1762
   macro avg       0.72      0.63      0.64      1762
weighted avg       0.73      0.74      0.71      1762



In [14]:
# Preprocess data
df.fillna("", inplace=True)
df['text'] = df['title'] + " " + df['cast'] + " " + df['country'] + " " + df['listed_in'] + " " + df['description']

# Features and target
X = df['text']
y = df['type']




In [92]:
len(set(i.split() for i in df.country.to_list()))

TypeError: unhashable type: 'list'

In [94]:
listik = []
for i in df.country.to_list():
    if len(i):
        listik += i.split(", ")

len(set(listik))

127

In [19]:
# Preprocess data
df.fillna("", inplace=True)
df['text'] = df['title'] + " " + df['cast'] + " " + df['country'] + " " + df['listed_in'] + " " + df['description']

# Features and target
X = df['text']
y = df['type']

# Text vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_vectorized = vectorizer.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.996594778660613

Classification Report:
               precision    recall  f1-score   support

       Movie       1.00      1.00      1.00      1214
     TV Show       1.00      0.99      0.99       548

    accuracy                           1.00      1762
   macro avg       1.00      1.00      1.00      1762
weighted avg       1.00      1.00      1.00      1762

